In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import unicodedata
import string
import re
import random

import time
import datetime
import math

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
from gensim.models.wrappers import FastText
import spacy
import numpy as np

#from redditscore.tokenizer import CrazyTokenizer

In [0]:

FASTTEXT_WORD2VEC = '/content/drive/My Drive/Colab Notebooks/cc.fr.300.bin'
french_model = FastText.load_fasttext_format(FASTTEXT_WORD2VEC)
weights_matrix = torch.zeros((len(SRC.vocab.itos), 300))
words_found = 0

for i, word in enumerate(SRC.vocab.itos):
    try: 
        word = word.lower()
        weights_matrix[i] = torch.FloatTensor(french_model[word])
        words_found += 1
    except KeyError:
        weights_matrix[i] = torch.rand((300,))


NameError: ignored

In [0]:
weights = torch.load("/content/drive/My Drive/Colab Notebooks/10320_fasttext_pt_word2vec.pt")

In [0]:
weights.shape

torch.Size([10320, 300])

In [0]:
manual_seed = 77
torch.manual_seed(manual_seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
n_gpu = torch.cuda.device_count()
if n_gpu > 0:
    torch.cuda.manual_seed(manual_seed)

cuda


In [0]:
!python -m spacy download en_core_web_sm
!python -m spacy download fr_core_news_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
     |████████████████████████████████| 14.7MB 408kB/s 
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.2.5-cp36-none-any.whl size=14727027 sha256=c470c8dce22f9e86e8ce06c5baa6ae937f931fe2469e6ff6c71763f70b9d940b
  Stored in directory: /tmp/pip-ephem-wheel-cache-sog9rxy9/wheels/46/1b/e6/29b020e3f9420a24c3f463343afe5136aaaf955dbc9e46dfc5
Successfully built fr-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [0]:

import fr_core_news_sm
import en_core_web_sm

spacy_fr = fr_core_news_sm.load()
spacy_en = en_core_web_sm.load()

In [0]:
import torchtext

In [0]:
def tokenize_fr(text):
    """
    Tokenizes French text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_fr.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [0]:
import pandas as pd
import csv

In [0]:
SRC = torchtext.data.Field(tokenize = tokenize_fr, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)
TRG = torchtext.data.Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

train, val, test = torchtext.data.TabularDataset.splits(
    path='/content/drive/My Drive/Colab Notebooks/MTNT/train', train='output.tsv',validation='output_val.tsv', test='output_test.tsv', 
    format='tsv', skip_header=True, fields=[('TRG', TRG), ('SRC', SRC)])

print(f"Number of training examples: {len(train.examples)}")
print(f"Number of validation examples: {len(val.examples)}")
print(f"Number of testing examples: {len(test.examples)}")

TRG.build_vocab(train, min_freq = 5)
SRC.build_vocab(train, min_freq = 5)

print(f"Unique tokens in source (fr) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

train_iter, val_iter, test_iter = torchtext.data.BucketIterator.splits(
    (train, val, test), # we pass in the datasets we want the iterator to draw data from
    batch_sizes=(16, 64, 64),device = device,
    sort_key=lambda x: len(x.SRC), # the BucketIterator needs to be told what function it should use to group the data.
    sort_within_batch=True)

Number of training examples: 18941
Number of validation examples: 875
Number of testing examples: 1021
Unique tokens in source (fr) vocabulary: 10320
Unique tokens in target (en) vocabulary: 8758


In [0]:

import pickle
with open("/content/drive/My Drive/Colab Notebooks/TRG.Field","wb")as f:
     pickle.dump(TRG,f)

with open("/content/drive/My Drive/Colab Notebooks/SRC.Field","wb")as f:
     pickle.dump(SRC,f)

In [0]:
print(vars(train.examples)['SRC'])

TypeError: ignored

In [0]:
print(vars(train.examples[-1])['SRC'])

['une', 'perpignanaise', 'vit', '12', 'ans', 'sans', 'eau', 'courante', ',', 'la', 'saur', 'condamnée', ' ', 'ces', '200', '€', 'ça', 'doit', 'plus', 'ou', 'moins', 'être', 'une', 'consommation', 'annuelle', '(', 'à', 'la', 'louche', '100m3', ')', ',', "c'", 'est', 'de', "l'", 'eau', 'pas', 'de', "l'", 'essence', '.']


In [0]:
def train_attn(model, iterator, optimizer, criterion, clip):
    manual_seed = 77
    torch.manual_seed(manual_seed)
    if n_gpu > 0:
        torch.cuda.manual_seed(manual_seed)
    
    model.train()
    
    epoch_loss = 0
    
    
    for i, batch in enumerate(iterator):
        
        src = batch.SRC
        trg = batch.TRG
        
        optimizer.zero_grad()
        
        output,_ = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

def evaluate_attn(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.SRC
            trg = batch.TRG

            output,_ = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

def init_weights(m):
    manual_seed = 77
    torch.manual_seed(manual_seed)
    if n_gpu > 0:
        torch.cuda.manual_seed(manual_seed)
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [0]:
def inference(model, file_name, src_vocab, trg_vocab, attention= False, max_trg_len = 64):
    '''
    Function for translation inference

    Input: 
    model: translation model;
    file_name: the directoy of test file that the first column is target reference, and the second column is source language;
    trg_vocab: Target torchtext Field
    attention: the model returns attention weights or not, default = False
    max_trg_len: the maximal length of translation text (optinal), default = 64

    Output:
    Corpus BLEU score.
    '''
    from nltk.translate.bleu_score import corpus_bleu
    from nltk.translate.bleu_score import sentence_bleu
    from torchtext.data import TabularDataset
    from torchtext.data import Iterator

    # convert index to text string
    def convert_itos(convert_vocab, token_ids):
        list_string = []
        for i in token_ids:
            if i == convert_vocab.vocab.stoi['<eos>']:
                break
            else:
                token = convert_vocab.vocab.itos[i]
                list_string.append(token)
        return list_string

        
    test = TabularDataset(
      path=file_name, # the root directory where the data lies
      format='tsv',
      skip_header=True, # if your tsv file has a header, make sure to pass this to ensure it doesn't get proceesed as data!
      fields=[('TRG', trg_vocab), ('SRC', src_vocab)])

    test_iter = Iterator(
    dataset = test, # we pass in the datasets we want the iterator to draw data from
    sort = False,batch_size=128,
    sort_key=None,
    shuffle=False,
    sort_within_batch=False,
    device = device,
    train=False
    )
  
    model.eval()
    all_trg = []
    all_translated_trg = []

    TRG_PAD_IDX = trg_vocab.vocab.stoi[trg_vocab.pad_token]

    with torch.no_grad():
    
        for i, batch in enumerate(test_iter):

            src = batch.SRC
            #src = [src len, batch size]

            trg = batch.TRG
            #trg = [trg len, batch size]

            batch_size = trg.shape[1]

            # create a placeholder for traget language with shape of [max_trg_len, batch_size] where all the elements are the index of <pad>. Then send to device
            trg_placeholder = torch.Tensor(max_trg_len, batch_size)
            trg_placeholder.fill_(TRG_PAD_IDX)
            trg_placeholder = trg_placeholder.long().to(device)
            if attention == True:
                output,_ = model(src, trg_placeholder, 0) #turn off teacher forcing
            else:
                output = model(src, trg_placeholder, 0) #turn off teacher forcing
            # get translation results, we ignor first token <sos> in both translation and target sentences. 
            # output_translate = [(trg len - 1), batch, output dim] output dim is size of target vocabulary.
            output_translate = output[1:]
            # store gold target sentences to a list 
            all_trg.append(trg[1:].cpu())

            # Choose top 1 word from decoder's output, we get the probability and index of the word
            prob, token_id = output_translate.data.topk(1)
            translation_token_id = token_id.squeeze(2).cpu()

            # store gold target sentences to a list 
            all_translated_trg.append(translation_token_id)
      
    all_gold_text = []
    all_translated_text = []
    for i in range(len(all_trg)): 
        cur_gold = all_trg[i]
        cur_translation = all_translated_trg[i]
        for j in range(cur_gold.shape[1]):
            gold_convered_strings = convert_itos(trg_vocab,cur_gold[:,j])
            trans_convered_strings = convert_itos(trg_vocab,cur_translation[:,j])

            all_gold_text.append(gold_convered_strings)
            all_translated_text.append(trans_convered_strings)

    corpus_all_gold_text = [[item] for item in all_gold_text]
    corpus_bleu_score = corpus_bleu(corpus_all_gold_text, all_translated_text)  
    return corpus_bleu_score

In [0]:
weights.shape

torch.Size([10320, 300])

In [0]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, n_layers, dropout):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dropout = dropout
        self.n_layers = n_layers

        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(weights))
        self.lstm = nn.LSTM(emb_dim, enc_hid_dim, n_layers, dropout=dropout, bidirectional = True)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
  
        embedded = self.dropout(self.embedding(src))
        
        outputs, (hidden, cell) = self.lstm(embedded)
       
        return outputs, (hidden, cell)


class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        #self.W_a = nn.Linear(enc_hid_dim + dec_hid_dim, dec_hid_dim)
        #self.v_a = nn.Parameter(torch.rand(dec_hid_dim))
        
    def forward(self, hidden, encoder_outputs):

        hidden = hidden[-1].unsqueeze(0).permute(1, 0, 2)
        
        encoder_outputs = encoder_outputs.permute(1, 2, 0)
        attention_vector = torch.bmm(hidden, encoder_outputs).squeeze(1)
        
        return F.softmax(attention_vector, dim=1)

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, dec_hid_dim, n_layers, dropout, attn):
        super().__init__()

        self.emb_dim = emb_dim
        self.output_dim = output_dim
        self.dec_hid_dim = dec_hid_dim
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.lstm = nn.LSTM(emb_dim, dec_hid_dim, n_layers, dropout=dropout, bidirectional = False)
        self.fc_out = nn.Linear(dec_hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

        self.f_mid = nn.Linear(dec_hid_dim * 2, dec_hid_dim)
        self.f_output = nn.Linear(dec_hid_dim, output_dim)

        self.attn = attn
        
    def forward(self, input, hidden, cell, encoder_outputs):
             
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        
      
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))

        attention_weights = self.attn(hidden, encoder_outputs)
        attention_weights = attention_weights.unsqueeze(1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        weighted = torch.bmm(attention_weights, encoder_outputs)
        weighted = weighted.permute(1, 0, 2)
        concat = torch.cat((hidden[-1].unsqueeze(0), weighted), dim = 2)
        concat = concat.squeeze(0)
        s_prime = self.f_mid(concat)
        prediction = self.f_output(s_prime)
        
        return prediction, hidden, cell, attention_weights

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):

        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        all_attention_weights = torch.zeros(trg.shape[1], trg.shape[0]-1, src.shape[0])

        encoder_outputs, (enc_hidden, enc_cell) = self.encoder(src)   #(4,8,512) (2, 8 ,1024)
        input = trg[0,:]

        #print('Enc_hidden', enc_hidden.shape)

        hidden1 = torch.cat((enc_hidden[0], enc_hidden[1]), dim = 1).unsqueeze(0)
        hidden2 = torch.cat((enc_hidden[2], enc_hidden[3]), dim = 1).unsqueeze(0)
        hidden = torch.cat((hidden1, hidden2), dim = 0)
        
        cell1 = torch.cat((enc_cell[0], enc_cell[1]), dim = 1).unsqueeze(0)
        cell2 = torch.cat((enc_cell[2], enc_cell[3]), dim = 1).unsqueeze(0)
        cell = torch.cat((cell1, cell2), dim = 0)
        for t in range(1, trg_len):
            
            output, hidden, cell, attention_weights = self.decoder(input, hidden, cell, encoder_outputs)
            
            all_attention_weights[:,t-1,:] = attention_weights.squeeze(1)

            outputs[t] = output

            teacher_force = random.random() < teacher_forcing_ratio

            top1 = output.argmax(1) 

            input = trg[t] if teacher_force else top1

        return outputs,all_attention_weights

In [0]:
INPUT_DIM = 10320
OUTPUT_DIM = 8758
ENC_EMB_DIM = 300
DEC_EMB_DIM = 300
N_LAYERS = 2
BI_DIRECTION = True
ENC_HID_DIM = 512
DEC_HID_DIM = 512 * 2 # maybe times 2?
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.3
TEACH_FORCING_RATE = 0.5
LEARNING_RT = 0.001
MAX_EPOCH = 20

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, N_LAYERS, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

optimizer = optim.Adam(model.parameters(),lr=LEARNING_RT)

def init_weights(m):
    manual_seed = 77
    torch.manual_seed(manual_seed)
    if n_gpu > 0:
        torch.cuda.manual_seed(manual_seed)
    for name, param in m.named_parameters():
        if 'encoder.embedding.weight' in name: 
            continue 
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        
        else:
            nn.init.constant_(param.data, 0)

model.apply(init_weights)

TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
print('<pad> token index: ',TRG_PAD_IDX)
## we will ignore the pad token in true target set
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

<pad> token index:  1


In [0]:
for name, param in model.named_parameters():
  print(name,param.shape)

encoder.embedding.weight torch.Size([10320, 300])
encoder.lstm.weight_ih_l0 torch.Size([2048, 300])
encoder.lstm.weight_hh_l0 torch.Size([2048, 512])
encoder.lstm.bias_ih_l0 torch.Size([2048])
encoder.lstm.bias_hh_l0 torch.Size([2048])
encoder.lstm.weight_ih_l0_reverse torch.Size([2048, 300])
encoder.lstm.weight_hh_l0_reverse torch.Size([2048, 512])
encoder.lstm.bias_ih_l0_reverse torch.Size([2048])
encoder.lstm.bias_hh_l0_reverse torch.Size([2048])
encoder.lstm.weight_ih_l1 torch.Size([2048, 1024])
encoder.lstm.weight_hh_l1 torch.Size([2048, 512])
encoder.lstm.bias_ih_l1 torch.Size([2048])
encoder.lstm.bias_hh_l1 torch.Size([2048])
encoder.lstm.weight_ih_l1_reverse torch.Size([2048, 1024])
encoder.lstm.weight_hh_l1_reverse torch.Size([2048, 512])
encoder.lstm.bias_ih_l1_reverse torch.Size([2048])
encoder.lstm.bias_hh_l1_reverse torch.Size([2048])
decoder.embedding.weight torch.Size([8758, 300])
decoder.lstm.weight_ih_l0 torch.Size([4096, 300])
decoder.lstm.weight_hh_l0 torch.Size([409

In [0]:
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(MAX_EPOCH):
    
    start_time = time.time()
    
    train_loss = train_attn(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate_attn(model, val_iter, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    # Create checkpoint at end of each epoch
    state_dict_model = model.state_dict() 
    state = {
        'epoch': epoch,
        'state_dict': state_dict_model,
        'optimizer': optimizer.state_dict()
        }

    torch.save(state, "/content/drive/My Drive/Colab Notebooks/model_"+str(epoch+1)+".pt")

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 8m 40s
	Train Loss: 5.717 | Train PPL: 303.942
	 Val. Loss: 5.932 |  Val. PPL: 376.820
Epoch: 02 | Time: 8m 45s
	Train Loss: 5.165 | Train PPL: 175.089
	 Val. Loss: 5.815 |  Val. PPL: 335.225
Epoch: 03 | Time: 8m 53s
	Train Loss: 4.817 | Train PPL: 123.645
	 Val. Loss: 5.648 |  Val. PPL: 283.644
Epoch: 04 | Time: 8m 39s
	Train Loss: 4.501 | Train PPL:  90.114
	 Val. Loss: 5.569 |  Val. PPL: 262.196
Epoch: 05 | Time: 8m 37s
	Train Loss: 4.233 | Train PPL:  68.957
	 Val. Loss: 5.613 |  Val. PPL: 274.027
Epoch: 06 | Time: 8m 35s
	Train Loss: 4.013 | Train PPL:  55.294
	 Val. Loss: 5.580 |  Val. PPL: 265.061
Epoch: 07 | Time: 8m 56s
	Train Loss: 3.814 | Train PPL:  45.322
	 Val. Loss: 5.471 |  Val. PPL: 237.697
Epoch: 08 | Time: 8m 55s
	Train Loss: 3.628 | Train PPL:  37.654
	 Val. Loss: 5.605 |  Val. PPL: 271.873
Epoch: 09 | Time: 8m 44s
	Train Loss: 3.445 | Train PPL:  31.355
	 Val. Loss: 5.723 |  Val. PPL: 305.773
Epoch: 10 | Time: 9m 2s
	Train Loss: 3.297 | Train PPL:

In [0]:
path = "/content/drive/My Drive/Colab Notebooks/MTNT/train/output.tsv"

tests = []
with open(path) as f:
  f = f.readlines()
  for i in range(3):
    print(f[765 + i].split("\t")[1])
    tests.append(f[765 + i].split("\t")[1])

A partir de ce moment la, j'arrive plus a contenir mes sentiments mais je sais que la distance fait que rien ne se passera et j'essaye de passer outre, mais j'y arrive pas.

A partir de la, c'est la descente aux enfers.

A partir de là il est facile de choisir les arguments les plus idiots et de les contredire.



In [0]:
INPUT_DIM = 10320
OUTPUT_DIM = 8758
ENC_EMB_DIM = 300
DEC_EMB_DIM = 300
N_LAYERS = 2
BI_DIRECTION = True
ENC_HID_DIM = 512
DEC_HID_DIM = 512 * 2 # maybe times 2?
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.3

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, N_LAYERS, DEC_DROPOUT, attn)

model_best = Seq2Seq(enc, dec, device).to(device)


In [0]:
model_best

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10320, 300)
    (lstm): LSTM(300, 512, num_layers=2, dropout=0.5, bidirectional=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(8758, 300)
    (lstm): LSTM(300, 1024, num_layers=2, dropout=0.3)
    (fc_out): Linear(in_features=1024, out_features=8758, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
    (f_mid): Linear(in_features=2048, out_features=1024, bias=True)
    (f_output): Linear(in_features=1024, out_features=8758, bias=True)
    (attn): Attention()
  )
)

In [0]:
model_best.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/model_6.pt')['state_dict'])

<All keys matched successfully>

In [0]:
tests = [['<sos>', 'a', 'partir', 'de', 'ce', 'moment', 'la', ',', "j'", 'arrive', 'plus', 'a', 'contenir', 'mes', 'sentiments', 'mais', 'je', 'sais', 'que', 'la', 'distance', 'fait', 'que', 'rien', 'ne', 'se', 'passera', 'et', "j'", 'essaye', 'de', 'passer', 'outre', ',', 'mais', "j'", 'y', 'arrive', 'pas', '.']]

In [0]:
# sents = [sentence_1, sentence_2, sentence_3]

ind = 1
for sent in tests:

  model_best.eval()
  src_token = SRC.preprocess(sent)
  print("src_token:", src_token)
  src_tensor = SRC.process([src_token],device=device)
  print("shape of source language: ", src_tensor.shape)

  trg_token = ['<pad>']*64
  trg_tensor = TRG.process([trg_token],device=device)

  output, all_attention_weights = model(src_tensor, trg_tensor,teacher_forcing_ratio = 0.0)
  output_dim = output.shape[-1] 
  output_translate = output[1:]

  source_language = src_tensor[:,0].cpu().numpy()
  translation_logit = output_translate[:,0,:].squeeze(1)
  weights = all_attention_weights[0,:,:].squeeze(1).cpu().detach().numpy()

  prob, token_id = translation_logit.data.topk(1)
  token_id = token_id.squeeze(1).cpu().numpy()

  src_language = []

  for i in source_language:
      if i == SRC.vocab.stoi['<eos>']:
          break
      else:
          token = SRC.vocab.itos[i]
          src_language.append(token)
  print("Source language:", src_language)

  trans_language = []

  for i in token_id:
      if i == TRG.vocab.stoi['<eos>']:
          break
      else:
          token = TRG.vocab.itos[i]
          trans_language.append(token)
  print("Our model translation: ", trans_language)

src_token: ['<sos>', 'a', 'partir', 'de', 'ce', 'moment', 'la', ',', "j'", 'arrive', 'plus', 'a', 'contenir', 'mes', 'sentiments', 'mais', 'je', 'sais', 'que', 'la', 'distance', 'fait', 'que', 'rien', 'ne', 'se', 'passera', 'et', "j'", 'essaye', 'de', 'passer', 'outre', ',', 'mais', "j'", 'y', 'arrive', 'pas', '.']
shape of source language:  torch.Size([42, 1])
Source language: ['<sos>', '<sos>', 'a', 'partir', 'de', 'ce', 'moment', 'la', ',', "j'", 'arrive', 'plus', 'a', 'contenir', 'mes', 'sentiments', 'mais', 'je', 'sais', 'que', 'la', 'distance', 'fait', 'que', 'rien', 'ne', 'se', 'passera', 'et', "j'", 'essaye', 'de', 'passer', 'outre', ',', 'mais', "j'", 'y', 'arrive', 'pas', '.']
Our model translation:  ['adults', 'bercy', 'conventional', 'condo', 'condo', 'sprays', 'sprays', 'modify', 'leaves', 'basic', 'course', 'leaves', '\t', 'vomit', 'trying', 'supermarkets', 'poll', 'actress', 'linking', 'christian', 'incompetence', 'desks', 'develop', 'frames', 'frames', 'bonus', 'bonus',